In [3]:
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

In [4]:
data_dir = 'C:/Users/raish/code/mrleyedataset'

In [5]:
transform = transforms.Compose([
    transforms.Grayscale(),  # Convert images to grayscale
    transforms.Resize((24, 24)),  # Resize all images to 24x24
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize between -1 and 1
])

In [6]:
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

In [7]:
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

In [8]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
class EyeStateModel(nn.Module):
    def __init__(self):
        super(EyeStateModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, 2)  # Output: [Open, Closed]
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 6 * 6)  # Flatten the tensor
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [10]:
model = EyeStateModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

In [12]:
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [14]:
train_model(model, train_loader, criterion, optimizer, num_epochs=5)

Epoch [1/5], Loss: 0.0927, Accuracy: 96.62%
Epoch [2/5], Loss: 0.0706, Accuracy: 97.40%
Epoch [3/5], Loss: 0.0588, Accuracy: 97.80%
Epoch [4/5], Loss: 0.0515, Accuracy: 98.09%
Epoch [5/5], Loss: 0.0453, Accuracy: 98.33%


In [15]:
test_model(model, test_loader)

Test Accuracy: 97.97%


In [16]:
torch.save(model.state_dict(), 'test_train_detect.pth')